# Data Analysis

## Part 1: Data Pre-Processing

In [168]:
import pandas as pd
import numpy as np
from datetime import datetime

### 1.1 Data Loading
Lets now load in the data from the saved .csv files and store each years data in a seperate dataframe.

In [169]:
# File path to 2021 sales data csv
file_path_2021 = "data/2021_property_sales_data.csv"
# Load the 2021 sales data into its own dataframe
df_2021 = pd.read_csv(file_path_2021)
#df_2021.head()

In [17]:
# File path to 2022 sales data csv
file_path_2022 = "data/2022_property_sales_data.csv"
# Load the 2022 sales data into its own dataframe
df_2022 = pd.read_csv(file_path_2022)
#df_2022.head()

In [16]:
# File path to 2023 sales data csv
file_path_2023 = "data/2023_property_sales_data.csv"
# Load the 2023 sales data into its own dataframe
df_2023 = pd.read_csv(file_path_2023)
#df_2023.head()

In [15]:
# File path to 2024 sales data csv
file_path_2024 = "data/2024_property_sales_data.csv"
# Load the 2024 sales data into its own dataframe
df_2024 = pd.read_csv(file_path_2024)
#df_2024.head()

### 1.2 Data Cleaning
Lets now determine the columns with missing oe null data.

In [19]:
df_2021.isnull().sum()

Unnamed: 0          0
Sale Date           0
Sale Price          0
Location            0
Year Built          0
Garden              0
Garage              0
Type                0
Style               0
Bedrooms            0
Bathrooms           0
First Time Buyer    0
dtype: int64

No columns have None on Nan as a value

### 1.3 Data Format
Lets now check if the values are of the same format by using frequency tables.

### Sale year
#### This data needs to all be of the format yyyy/mm/dd
Lets test for this and perform some cleaning.

In [170]:
df_2021["Sale Date"].value_counts()

Sale Date
Sold 15 January 2021    20
Sold 2021-03-02         20
Sold 2021-03-27         20
Sold 2021-04-27         20
Sold 2021-05-08         20
Sold 2021-05-24         20
Sold 2021-05-30         20
Sold 2021-06-07         20
Sold 2021-06-15         20
Sold 2021-07-03         20
Sold 2021-07-13         20
Sold 2021-07-26         20
Sold 2021-08-02         20
Sold 2021-08-16         20
Sold 2021-10-04         20
Sold 2021-11-24         20
Sold 2021-12-19          3
Name: count, dtype: int64

In [176]:
## Helper function to parse date of format "15 January 2021" into 2021-01-15
def convert_date(date_str):
    try:
        datetime.strptime(date_str, "%Y-%m-%d")
        return date_str
    except ValueError:
        pass
    try:
        date_obj = datetime.strptime(date_str, "%d %B %Y")
        return date_obj.strftime("%Y-%m-%d")
    except ValueError:
        return None

Lets clean the date values to be of the format "yyyy-mm-dd"

In [177]:
df_2021["Sale Date"] = df_2021["Sale Date"].str.strip().str.replace("Sold", "",regex=False).str.strip()
df_2021["Sale Date"] = df_2021["Sale Date"].apply(convert_date)

Lets make double check all values are of the same format.

In [179]:
df_2021["Sale Date"].value_counts()

Sale Date
2021-01-15    20
2021-03-02    20
2021-03-27    20
2021-04-27    20
2021-05-08    20
2021-05-24    20
2021-05-30    20
2021-06-07    20
2021-06-15    20
2021-07-03    20
2021-07-13    20
2021-07-26    20
2021-08-02    20
2021-08-16    20
2021-10-04    20
2021-11-24    20
2021-12-19     3
Name: count, dtype: int64

### Sale Price
#### This data should be of the format float
Lets test for this and perform some cleaning.

In [180]:
df_2021["Sale Price"].value_counts()

Sale Price
€359,086.00    1
€363,337.00    1
€448,551.00    1
€715,386.00    1
€566,533.00    1
              ..
€ 311,600      1
€141,336.00    1
€192,553.00    1
€370,181.00    1
€574,519.00    1
Name: count, Length: 323, dtype: int64

This data is formatted with a € as a prefix and entries are a mix of string representations of floats and ints.
 Lets clean this data to remove the currency prefix, the "," symbol, and format all entries as a string rounded to 2 decimal places.

In [182]:
# Helper function to remove currenct sign and format the values as a 
# string with format double rounded to 2 decimal places.

def clean_sale_prices(value):
    # Remove the currency sign
    value = str(value).replace("€", "").replace(",", "").strip()

    try:
        # Try to convert the string to a float and round to 2 decimal places
        rounded = round(float(value), 2)
        # Return a string rounded to 2 decimal places
        return f"{rounded:.2f}" 
    except ValueError:
        return None

Apply the helper function to all entries in the column.

In [183]:
df_2021["Sale Price"] = df_2021["Sale Price"].apply(clean_sale_prices)

Lets double check the output of our new column.

In [184]:
df_2021["Sale Price"].value_counts()

Sale Price
359086.00    1
363337.00    1
448551.00    1
715386.00    1
566533.00    1
            ..
311600.00    1
141336.00    1
192553.00    1
370181.00    1
574519.00    1
Name: count, Length: 323, dtype: int64

### Location
#### This data should be a valid string format.
Lets test for this and perform some cleaning.

In [185]:
df_2021["Location"].value_counts()

Location
Oak Park         83
Brookville       52
Rivermount       42
Avoca            36
Porter Square    33
Broomhouse       28
Summerston       25
Beacon Hill      24
Name: count, dtype: int64

All data is of a valid format.

### Year Built
#### This data should be of the format int
Lets test for this and perform some cleaning.
Some years have chars before and after the year so lets remove these.
Years with "Unknown" in the column will non become Nan.

In [186]:
freq_table = df_2021["Year Built"].value_counts()
#print(freq_table.to_string())

In [187]:
# extract the first 4 digits from anywhere in the string, and add back into the column.
df_2021["Year Built"] = df_2021["Year Built"].str.extract(r"(\d{4})")

In [190]:
cleaned_frequency_table = df_2021["Year Built"].value_counts()
#print(cleaned_frequency_table.to_string())

### Gardern
#### This data should be of the format, either yes or no
Lets test for this and perform some cleaning.

In [191]:
df_2021["Garden"].value_counts()

Garden
Yes    299
No      19
???      5
Name: count, dtype: int64

Lets replace entries with "???" with "Unknown"

In [193]:
map1 = {"???":"Unknown"}
df_2021["Garden"] = df_2021["Garden"].replace(map1)

In [194]:
df_2021["Garden"].value_counts()

Garden
Yes        299
No          19
Unknown      5
Name: count, dtype: int64

### Garage
#### This data should be of the format, either yes or no.
Lets test for this and perform some cleaning.

In [195]:
df_2021["Garage"].value_counts()

Garage
Yes    184
No     120
???     19
Name: count, dtype: int64

Lets replace entries of the format "???" with "Unknown"

In [196]:
df_2021["Garage"] = df_2021["Garage"].replace(map1)

In [197]:
df_2021["Garage"].value_counts()

Garage
Yes        184
No         120
Unknown     19
Name: count, dtype: int64

### Type
#### This data should be a valid house description  string, (detached, terrace etc)
Lets test for this and perform some cleaning.

In [198]:
df_2021["Type"].value_counts()

Type
Detached          238
Bungalow           36
Det.               11
Semi-Detached      11
Duplex             10
End-of-Terrace      8
Terraced            7
Semi-D              2
Name: count, dtype: int64

A mix of text is shown here, some entries mean the same thing but are formatted different (e.g Semi-D = Semi-Detached) lets map simular values to be added accumulated corrextly.

In [199]:
house_type_map = {"Det.":"Detached", "Semi-D":"Semi-Detached"}
df_2021["Type"] = df_2021["Type"].str.strip()
df_2021["Type"] = df_2021["Type"].replace(house_type_map)

In [200]:
s = df_2021["Type"].value_counts()
print(s.to_string())

Type
Detached          249
Bungalow           36
Semi-Detached      13
Duplex             10
End-of-Terrace      8
Terraced            7


### Style
#### This data should be a valid house style description 1,2,3 stories.
Lets test for this and perform some cleaning.

In [201]:
df_2021["Style"].value_counts()

Style
1-Storey      205
2-Storey       94
1.5-Storey     24
Name: count, dtype: int64

All data is formatted correctly here!

### Bedrooms
#### This data should be a valid int.
Lets test for this and perform some cleaning.

In [202]:
df_2021["Bedrooms"].value_counts()

Bedrooms
3    198
2     68
4     42
1     10
5      5
Name: count, dtype: int64

Again all data is correctly formatted here.

### Bathrooms
#### This data should be a valid int.
Lets test for this and perform some cleaning.

In [203]:
df_2021["Bathrooms"].value_counts()

Bathrooms
1    170
2    146
3      7
Name: count, dtype: int64

Again all data is formatted correctly here.

### First Time Buyer
#### This data should be a string either yes or no.
Lets test for this and perform some cleaning.

In [41]:
df_2021["First Time Buyer"].value_counts()

First Time Buyer
No     211
Yes     94
NO      18
Name: count, dtype: int64

This data needs needs to be cleaned so strings are calculated as equal regardsless of upper and lowercase entries.

In [204]:
first_time_buyer_map = {"YES": "Yes", "NO": "No"}
df_2021["First Time Buyer"] = df_2021["First Time Buyer"].str.strip()
df_2021["First Time Buyer"] = df_2021["First Time Buyer"].replace(first_time_buyer_map)

Lets quickly verify our new data is now formatted correctly.

In [205]:
df_2021["First Time Buyer"].value_counts()

First Time Buyer
No     229
Yes     94
Name: count, dtype: int64

### Our 2021 Dataset is now preprocessed lets now save it as a cleaned CSV.

In [206]:
# Maybe save the 2021 data here.